# 1) Import needed libraries

In [1]:
#!pip install BeautifulSoup4 
from bs4 import BeautifulSoup
#!conda install -c anaconda lxml --yes
import requests # library to handle requests
import pandas as pd

# 2) Aquire the webpage containing the information we need

In [2]:

source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup = BeautifulSoup(source, 'lxml')

# 3) Seperate the table of interest into a dataframe

In [3]:

table = soup.table
Toronto_Neigh = pd.read_html(str(table), header=0)[0]

Toronto_Neigh.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


# 4) Remove postcodes with unassigned boroughs

In [4]:
for index, row in Toronto_Neigh.iterrows():
    if row['Borough']=="Not assigned":
        Toronto_Neigh = Toronto_Neigh.drop(index)
Toronto_Neigh.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


# 5) Replace neighbourhood with borough name for postcodes with no assigned neighbourhood info

In [5]:
for index, row in Toronto_Neigh.iterrows():
    if row['Neighbourhood']=="Not assigned":
        row['Neighbourhood'] = row['Borough']
Toronto_Neigh.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


# 6) Group neighbourhoods based on postcodes

In [6]:
Toronto_Neigh = Toronto_Neigh.groupby(['Postcode','Borough'])['Neighbourhood'].apply(','.join)
Toronto_Neigh = pd.DataFrame(Toronto_Neigh).reset_index()

# Here is the resulting data frame and its shape

In [7]:
Toronto_Neigh.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [8]:
Toronto_Neigh.shape

(103, 3)